In [ ]:
import cv2
import os
import csv
import numpy as np
from skimage import feature

def extract_lbp_features(image_path, radius=1, n_points=8):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    lbp = feature.local_binary_pattern(image, n_points, radius, method="uniform")
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    return hist

def load_images_and_extract_features(directory):
    feature_list = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg"):
            image_path = os.path.join(directory, filename)
            features = extract_lbp_features(image_path)
            feature_list.append(features)
    return feature_list

def save_features_to_csv(data, labels, csv_filename):
    with open(csv_filename, 'w', newline='') as file:
        writer = csv.writer(file)
        for label, features in zip(labels, data):
            writer.writerow([label] + list(features))

face_dir = "./data/face/train/"
face_csv = "train_face.csv"
face_features = load_images_and_extract_features(face_dir)
face_labels = [1] * len(face_features)  # Label 1 for face
save_features_to_csv(face_features, face_labels, face_csv)
print(f"LBP features saved to {face_csv}")

val_dir = "./data/face/val/"
val_csv = "val_face.csv"
val_features = load_images_and_extract_features(val_dir)
val_labels = [1] * len(val_features)  # Label 1 for face
save_features_to_csv(val_features, val_labels, val_csv)
print(f"LBP features saved to {val_csv}")

test_dir = "./data/face/test/"
test_csv = "test_face.csv"
test_features = load_images_and_extract_features(test_dir)
test_labels = [1] * len(test_features)  # Label 1 for face
save_features_to_csv(test_features, test_labels, test_csv)
print(f"LBP features saved to {test_csv}")

In [ ]:
import numpy as np

def read_csv(filename):
    data = []
    with open(filename, 'r') as file:
        for line in file:
            parts = line.strip().split(',')
            data.append([int(parts[0])] + [float(x) for x in parts[1:]])
    return data

train_data = read_csv("train_face.csv")
val_data = read_csv("val_face.csv")
test_data = read_csv("test_face.csv")

X_train = np.array([row[1:] for row in train_data])
y_train = np.array([row[0] for row in train_data])
X_val = np.array([row[1:] for row in val_data])
y_val = np.array([row[0] for row in val_data])
X_test = np.array([row[1:] for row in test_data])
y_test = np.array([row[0] for row in test_data])

def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2) ** 2))

def predict_knn(X_train, y_train, x, k=3):
    distances = [euclidean_distance(x, x_train) for x_train in X_train]
    k_indices = np.argsort(distances)[:k]
    k_nearest_labels = [y_train[i] for i in k_indices]
    most_common = np.bincount(k_nearest_labels).argmax()
    return most_common

class LinearSVM:
    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=1000):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.w = None
        self.b = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)
        self.b = 0

        for _ in range(self.n_iters):
            for idx, x in enumerate(X):
                condition = y[idx] * (np.dot(x, self.w) - self.b) >= 1
                if condition:
                    self.w -= self.lr * (2 * self.lambda_param * self.w)
                else:
                    self.w -= self.lr * (2 * self.lambda_param * self.w - np.dot(x, self.w))
                    self.b -= self.lr * y[idx]

    def predict(self, X):
        approx = np.dot(X, self.w) - self.b
        return np.sign(approx)

best_knn_accuracy = 0
best_knn_model = None
best_svm_accuracy = 0
best_svm_model = None

for k in [1, 3, 5, 7]:
    knn_predictions = [predict_knn(X_train, y_train, x, k) for x in X_val]
    knn_accuracy = (np.array(knn_predictions) == y_val).mean()

    if knn_accuracy > best_knn_accuracy:
        best_knn_accuracy = knn_accuracy
        best_knn_model = k

svm = LinearSVM()
svm.fit(X_train, y_train)
svm_predictions = svm.predict(X_val)
svm_accuracy = (np.array(svm_predictions) == y_val).mean()

if svm_accuracy > best_svm_accuracy:
    best_svm_accuracy = svm_accuracy
    best_svm_model = svm

knn_test_predictions = [predict_knn(X_train, y_train, x, best_knn_model) for x in X_test]
svm_test_predictions = best_svm_model.predict(X_test)

knn_test_accuracy = (np.array(knn_test_predictions) == y_test).mean()
svm_test_accuracy = (np.array(svm_test_predictions) == y_test).mean()

print(f"Best KNN Model Accuracy on Val.csv: {best_knn_accuracy:.2f}")
print(f"Accuracy on Test.csv using Best KNN Model: {knn_test_accuracy:.2f}")

print(f"Best SVM Model Accuracy on Val.csv: {best_svm_accuracy:.2f}")
print(f"Accuracy on Test.csv using Best SVM Model: {svm_test_accuracy:.2f}")

In [ ]:
import os
import numpy as np
from sklearn import svm
from sklearn.metrics import accuracy_score, classification_report
import csv

def extract_lbp_features(image_path, radius=1, n_points=8):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    lbp = feature.local_binary_pattern(image, n_points, radius, method="uniform")
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    return hist

def load_images_and_extract_features(directory):
    feature_list = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg"):
            image_path = os.path.join(directory, filename)
            features = extract_lbp_features(image_path)
            feature_list.append(features)
    return feature_list

def save_features_to_csv(data, labels, csv_filename):
    with open(csv_filename, 'w', newline='') as file:
        writer = csv.writer(file)
        for label, features in zip(labels, data):
            writer.writerow([label] + list(features))

non_face_dir = "./data/non-face/"
non_face_csv = "train_non_face.csv"
non_face_features = load_images_and_extract_features(non_face_dir)
non_face_labels = [0] * len(non_face_features)

save_features_to_csv(non_face_features, non_face_labels, non_face_csv)
print(f"LBP features saved to {non_face_csv}")

X_face = np.genfromtxt("train_face.csv", delimiter=',')
X_non_face = np.genfromtxt("train_non_face.csv", delimiter=',')

y_face = np.ones(X_face.shape[0])
y_non_face = np.zeros(X_non_face.shape[0])

X = np.vstack((X_face, X_non_face))
y = np.concatenate((y_face, y_non_face))

np.random.seed(42)
n_samples = X.shape[0]
n_test = int(n_samples * 0.2)
test_indices = np.random.choice(n_samples, n_test, replace=False)
train_indices = np.array(list(set(range(n_samples)) - set(test_indices)))
X_train, X_test = X[train_indices], X[test_indices]
y_train, y_test = y[train_indices], y[test_indices]

svm_classifier = svm.SVC(kernel='linear')
svm_classifier.fit(X_train, y_train)

y_pred = svm_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print(report)